## **Environment Setup and Data Input**

In [ ]:
install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
BiocManager::install("gage")
BiocManager::install("gageData")
BiocManager::install("pathview")
BiocManager::install("clusterProfiler")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'gage'”
Old packages: 'bit'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'gageData'”
Old packages: 'bit'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositorie

In [ ]:
#Load libraries
library(gage)
library(gageData)
library(pathview)
library(clusterProfiler)

In [ ]:
# Set the desired organism
organism = "org.Hs.eg.db"
BiocManager::install(organism, character.only = TRUE)
library(organism, character.only = TRUE)

# Load DESeq2 results file containing ENSEMBL IDs
result <- read.table("/content/DESeq2_Result (3).txt", sep="\t", header = TRUE, row.names=1)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'org.Hs.eg.db'”
Old packages: 'bit'



## **Data Preparation and Gene ID Mapping**

In [ ]:
# Check top 10 lines in results
head(result)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000290825,0.3706110,-0.2230679,1.3558050,-0.1645280,8.693155e-01,NA
ENSG00000227232,570.5388616,-0.6921837,0.1529257,-4.5262745,6.003260e-06,4.982777e-05
ENSG00000278267,16.3868742,-0.6351872,0.3223845,-1.9702786,4.880645e-02,1.156835e-01
ENSG00000243485,0.9152847,-1.3016060,0.8976245,-1.4500563,1.470428e-01,2.738303e-01
ENSG00000237613,0.3996788,-1.6900154,1.3112132,-1.2888944,1.974348e-01,NA
ENSG00000238009,6.1618180,-0.2373482,0.3293324,-0.7206949,4.710973e-01,6.315650e-01


In [ ]:
# List of KEGG pathways and significant pathways for Homo sapiens
data(kegg.sets.hs)
data(sigmet.idx.hs)

# Filter important KEGG pathways
kegg.sets.hs <- kegg.sets.hs[sigmet.idx.hs]

In [ ]:
# Convert gene IDs for gseKEGG function
# We will lose some genes here because not all IDs will be converted
ids <-bitr(rownames(result), fromType = "ENSEMBL", toType = "ENTREZID", OrgDb=organism)

# remove duplicate IDS (here I use "ENSEMBL", but it should be whatever was selected as keyType)
dedup_ids = ids[!duplicated(ids[c("ENSEMBL")]),]

# Create a new dataframe df2 which has only the genes which were successfully mapped using the bitr function above
res_dedup = result[dedup_ids$ENSEMBL,]

# Create a new column in df2 with the corresponding ENTREZ IDs
res_dedup$ENTREZID = dedup_ids$ENTREZID

#Extract the fold changes from the results
foldchanges <- res_dedup$log2FoldChange

# Name fold change vector with ENTREZ ids
names(foldchanges) <- res_dedup$ENTREZID

# omit any NA values
foldchanges<-na.omit(foldchanges)

# sort the list in decreasing order (required for clusterProfiler)
foldchanges = sort(foldchanges, decreasing = TRUE)

#check top 5 values
head(foldchanges)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(rownames(result), fromType = "ENSEMBL", toType = "ENTREZID", :
“35.2% of input gene IDs are fail to map...”


54490     28785     28891     28454     28477     28455 
22.358655  9.011919  8.438300  8.233993  8.055829  7.988895

In [ ]:
# Extract log2 fold changes and assign ENTREZ IDs
foldchanges <- res_dedup$log2FoldChange
names(foldchanges) <- res_dedup$ENTREZID

# Remove NA values from foldchanges and sort
foldchanges <- na.omit(foldchanges)
foldchanges <- sort(foldchanges, decreasing=TRUE)

# Check top fold change values
head(foldchanges)

54490     28785     28891     28454     28477     28455 
22.358655  9.011919  8.438300  8.233993  8.055829  7.988895

## **Pathway Analysis with GAGE**

In [ ]:
# Check overlap between your data and KEGG pathway genes
overlapping_genes <- intersect(names(foldchanges), unlist(kegg.sets.hs))
cat("Number of overlapping genes with KEGG pathways: ", length(overlapping_genes), "\n")

Number of overlapping genes with KEGG pathways:  3706 


In [ ]:
# Perform GAGE pathway analysis if enough overlapping genes
if (length(overlapping_genes) > 10) {
    kegg_result <- gage(foldchanges, gsets=kegg.sets.hs, same.dir=TRUE)

    # Write results of all pathways (upregulated and downregulated) to a file
    if (!is.null(kegg_result)) {
        write.table(kegg_result, file="all_pathways.txt", sep="\t", row.names=TRUE, quote=FALSE)
    } else {
        cat("No significant pathways found.\n")
    }
} else {
    cat("Not enough overlapping genes to perform pathway analysis.\n")
}

In [ ]:
# Extract upregulated and downregulated pathways
Upreg_pathways <- data.frame(Pathway_id=rownames(kegg_result$greater), kegg_result$greater)
Downreg_pathways <- data.frame(Pathway_id=rownames(kegg_result$less), kegg_result$less)

# Check for any NA pathway names
if (any(is.na(Upreg_pathways$Pathway_id))) {
    cat("Some upregulated pathways are missing names.\n")
}
if (any(is.na(Downreg_pathways$Pathway_id))) {
    cat("Some downregulated pathways are missing names.\n")
}

## **Pathway Gene Extraction**

In [ ]:
# Function to extract genes involved in each pathway
extract_pathway_genes <- function(pathway_data, pathway_name, kegg_sets, foldchanges) {
    pathway_genes <- kegg_sets[[pathway_name]]
    pathway_genes <- intersect(pathway_genes, names(foldchanges))
    return(paste(pathway_genes, collapse = ", "))
}

## **Enrichment Analysis**

In [ ]:
# Load necessary libraries
library(clusterProfiler)
library(org.Hs.eg.db)

# Function to extract genes involved in each pathway and convert to gene symbols and calculate Enrichment Score
extract_pathway_genes_and_enrichment <- function(pathway_name, kegg_sets, foldchanges) {
    # Print the pathway name being processed for debugging
    cat("Processing pathway:", pathway_name, "\n")

    # Ensure pathway_name is in the correct format to match kegg_sets keys
    pathway_genes <- kegg_sets[[pathway_name]]

    # If the pathway is not found, print warning and return NA
    if (is.null(pathway_genes)) {
        cat("Warning: Pathway", pathway_name, "not found in KEGG sets\n")
        return(list(Genes = NA, EnrichmentScore = NA))
    }

    # Extract genes that are in both the pathway and the foldchanges list
    overlapping_genes <- intersect(pathway_genes, names(foldchanges))

    # If no genes are found, return NA
    if (length(overlapping_genes) == 0) {
        return(list(Genes = NA, EnrichmentScore = NA))
    }

    # Calculate observed DEGs (overlapping genes)
    observed_DEGs <- length(overlapping_genes)

    # Calculate expected genes (total genes in the pathway)
    total_genes <- length(pathway_genes)

    # Calculate the enrichment score
    enrichment_score <- observed_DEGs / total_genes

    # Convert ENTREZ IDs to gene symbols
    gene_symbols <- bitr(overlapping_genes, fromType = "ENTREZID", toType = "SYMBOL", OrgDb = org.Hs.eg.db)

    # Return the gene symbols and enrichment score as a list
    if (nrow(gene_symbols) > 0) {
        genes <- paste(gene_symbols$SYMBOL, collapse = ", ")
    } else {
        genes <- NA
    }

    return(list(Genes = genes, EnrichmentScore = enrichment_score))
}

# Extract pathway IDs from the results
upreg_pathway_ids <- rownames(Upreg_pathways)
downreg_pathway_ids <- rownames(Downreg_pathways)

# Add a column for genes affected in upregulated pathways (as gene symbols) and Enrichment Score
upreg_results <- lapply(upreg_pathway_ids, extract_pathway_genes_and_enrichment, kegg_sets = kegg.sets.hs, foldchanges = foldchanges)

# Ensure all elements in the result contain 'Genes' and 'EnrichmentScore'
Upreg_pathways$Genes <- sapply(upreg_results, function(x) if (!is.null(x$Genes)) x$Genes else NA)
Upreg_pathways$EnrichmentScore <- sapply(upreg_results, function(x) if (!is.null(x$EnrichmentScore)) x$EnrichmentScore else NA)

# Add a column for genes affected in downregulated pathways (as gene symbols) and Enrichment Score
downreg_results <- lapply(downreg_pathway_ids, extract_pathway_genes_and_enrichment, kegg_sets = kegg.sets.hs, foldchanges = foldchanges)

# Ensure all elements in the result contain 'Genes' and 'EnrichmentScore'
Downreg_pathways$Genes <- sapply(downreg_results, function(x) if (!is.null(x$Genes)) x$Genes else NA)
Downreg_pathways$EnrichmentScore <- sapply(downreg_results, function(x) if (!is.null(x$EnrichmentScore)) x$EnrichmentScore else NA)

# Print the first few rows to check if gene symbols and enrichment scores are being added correctly
print(head(Upreg_pathways))
print(head(Downreg_pathways))

Processing pathway: hsa04062 Chemokine signaling pathway 


'select()' returned 1:1 mapping between keys and columns



Processing pathway: hsa04640 Hematopoietic cell lineage 


'select()' returned 1:1 mapping between keys and columns



Processing pathway: hsa04672 Intestinal immune network for IgA production 


...

Processing pathway: hsa00780 Biotin metabolism 


'select()' returned 1:1 mapping between keys and columns



Processing pathway: hsa00072 Synthesis and degradation of ketone bodies 


'select()' returned 1:1 mapping between keys and columns



                                                                                                 Pathway_id
hsa04062 Chemokine signaling pathway                                   hsa04062 Chemokine signaling pathway
hsa04640 Hematopoietic cell lineage                                     hsa04640 Hematopoietic cell lineage
hsa04672 Intestinal immune network for IgA production hsa04672 Intestinal immune network for IgA production
hsa04650 Natural killer cell mediated cytotoxicity       hsa04650 Natural killer cell mediated cytotoxicity
hsa04612 Antigen processing and presentation                   hsa04612 Antigen processing and presentation
hsa04620 Toll-like receptor signaling pathway                 hsa04620 Toll-like receptor signaling pathway
                                                         p.geomean stat.mean
hsa04062 Chemokine signaling pathway                  6.940839e-07  4.944564
hsa04640 Hematopoietic cell lineage                   9.972178e-07  5.001760
hsa04672 Inte

In [ ]:
# Write the results to files
write.table(Upreg_pathways, file = "upreg_pathways_enrichment.txt", sep = "\t", row.names = FALSE, quote = FALSE)
write.table(Downreg_pathways, file = "downreg_pathways_enrichment.txt", sep = "\t", row.names = FALSE, quote = FALSE)

In [ ]:
# Extract the top 5 upregulated pathways
keggrespathways_up <- rownames(kegg_result$greater)[1:5]

# Extract the IDs of the top 5 upregulated pathways
keggresids_up <- substr(keggrespathways_up, start=1, stop=8)

# Write the IDs of the top 5 upregulated pathways to a file
write.table(keggresids_up, file="top5_upreg_pathways_ids.txt", sep="\t", row.names=FALSE, quote=FALSE)

# Draw pathway maps for the top 5 upregulated pathways
pathview(gene.data=foldchanges, pathway.id=keggresids_up, species="hsa")

# Extract the top 5 downregulated pathways
keggrespathways_down <- rownames(kegg_result$less)[1:5]

# Extract the IDs of the top 5 downregulated pathways
keggresids_down <- substr(keggrespathways_down, start=1, stop=8)

# Write the IDs of the top 5 downregulated pathways to a file
write.table(keggresids_down, file="top5_downreg_pathways_ids.txt", sep="\t", row.names=FALSE, quote=FALSE)

# Draw pathway maps for the top 5 downregulated pathways
pathview(gene.data=foldchanges, pathway.id=keggresids_down, species="hsa")


'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04062.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04640.pathview.png

Info: Downloading xml files for hsa04672, 1/1 pathways..

Info: Downloading png files for hsa04672, 1/1 pathways..

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04672.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04650.pathview.png

Info: Downloading xml files for hsa04612, 1/1 pathways..

Info: Downloading png files for hsa04612, 1/1 pathways..

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04612.pathview.png

Info: Downloading xml files for hsa04530, 1/1 pathway

In [ ]:
library(pathview)

# Extract the top 5 upregulated pathways
keggrespathways_up <- rownames(kegg_result$greater)[1:5]

# Extract the IDs of the top 5 upregulated pathways
keggresids_up <- substr(keggrespathways_up, start=1, stop=8)

# Write the IDs of the top 5 upregulated pathways to a file
write.table(keggresids_up, file="top5_upreg_pathways_ids.txt", sep="\t", row.names=FALSE, quote=FALSE)

# Save pathway maps for the top 5 upregulated pathways with 300 DPI
for (pathway in keggresids_up) {
  png(filename = paste0("pathway_up_", pathway, ".png"), width = 2000, height = 2000, res = 300)
  pathview(gene.data = foldchanges, pathway.id = pathway, species = "hsa")
  dev.off()
}

# Extract the top 5 downregulated pathways
keggrespathways_down <- rownames(kegg_result$less)[1:5]

# Extract the IDs of the top 5 downregulated pathways
keggresids_down <- substr(keggrespathways_down, start=1, stop=8)

# Write the IDs of the top 5 downregulated pathways to a file
write.table(keggresids_down, file="top5_downreg_pathways_ids.txt", sep="\t", row.names=FALSE, quote=FALSE)

# Save pathway maps for the top 5 downregulated pathways with 300 DPI
for (pathway in keggresids_down) {
  png(filename = paste0("pathway_down_", pathway, ".png"), width = 2000, height = 2000, res = 300)
  pathview(gene.data = foldchanges, pathway.id = pathway, species = "hsa")
  dev.off()
}


'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04062.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04640.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04672.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04650.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04612.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04530.pathview.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04920.pathview.png


In [ ]:
# Load necessary library
library(pathview)

# Extract the top 5 upregulated pathways
keggrespathways_up <- rownames(kegg_result$greater)[1:5]

# Extract the IDs of the top 5 upregulated pathways
keggresids_up <- substr(keggrespathways_up, start=1, stop=8)

# Write the IDs of the top 5 upregulated pathways to a file
write.table(keggresids_up, file="top5_upreg_pathways_ids.txt", sep="\t", row.names=FALSE, quote=FALSE)

# Save pathway maps for the top 5 upregulated pathways with increased dimensions
for (pathway in keggresids_up) {
  pathview(gene.data = foldchanges,
           pathway.id = pathway,
           species = "hsa",
           kegg.native = TRUE,
           out.suffix = "upregulated",
           high.res = TRUE)  # Enable high-resolution output
}

# Extract the top 5 downregulated pathways
keggrespathways_down <- rownames(kegg_result$less)[1:5]

# Extract the IDs of the top 5 downregulated pathways
keggresids_down <- substr(keggrespathways_down, start=1, stop=8)

# Write the IDs of the top 5 downregulated pathways to a file
write.table(keggresids_down, file="top5_downreg_pathways_ids.txt", sep="\t", row.names=FALSE, quote=FALSE)

# Save pathway maps for the top 5 downregulated pathways with increased dimensions
for (pathway in keggresids_down) {
  pathview(gene.data = foldchanges,
           pathway.id = pathway,
           species = "hsa",
           kegg.native = TRUE,
           out.suffix = "downregulated",
           high.res = TRUE)  # Enable high-resolution output
}


'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04062.upregulated.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04640.upregulated.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04672.upregulated.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04650.upregulated.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04612.upregulated.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04530.downregulated.png

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hs

In [ ]:
# Visualize one upregulated pathway
pathview(gene.data=foldchanges, pathway.id="hsa04710")

# Visualize one downregulated pathway
pathview(gene.data=foldchanges, pathway.id="hsa04713")

# Obtain a PDF-based output of pathway visualization
pathview(gene.data=foldchanges, pathway.id="hsa04514", kegg.native=FALSE)

Info: Downloading xml files for hsa04710, 1/1 pathways..

Info: Downloading png files for hsa04710, 1/1 pathways..

Argument gene.idtype or cpd.idtype may be wrong.

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04710.pathview.png

Info: Downloading xml files for hsa04713, 1/1 pathways..

Info: Downloading png files for hsa04713, 1/1 pathways..

Argument gene.idtype or cpd.idtype may be wrong.

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04713.pathview.png

Info: Downloading xml files for hsa04514, 1/1 pathways..

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04514.pathview.pdf



In [ ]:
# Load necessary library
library(pathview)

# List of pathway IDs
pathway_ids <- c("hsa04062", "hsa00140", "hsa00830", "hsa00100", "hsa04620", "hsa04380")

# Iterate through each pathway ID and create high-resolution PNG plots
for (pathway in pathway_ids) {
  pathview(gene.data = foldchanges,
           pathway.id = pathway,
           species = "hsa",
           kegg.native = TRUE,  # Use native KEGG pathway view
           out.suffix = pathway,  # Suffix for output files
           high.res = TRUE)  # Enable high-resolution output
}




'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa04062.hsa04062.png

Info: Downloading xml files for hsa00140, 1/1 pathways..

Info: Downloading png files for hsa00140, 1/1 pathways..

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa00140.hsa00140.png

Info: Downloading xml files for hsa00830, 1/1 pathways..

Info: Downloading png files for hsa00830, 1/1 pathways..

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa00830.hsa00830.png

Info: Downloading xml files for hsa00100, 1/1 pathways..

Info: Downloading png files for hsa00100, 1/1 pathways..

'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /content

Info: Writing image file hsa00100.hsa00100.png

Info: Downloading xml files for hsa04620, 1/1 pathways..

Info: Downloading png 

In [ ]:
# Optionally, save one pathway as a PDF for better scalability
pathview(gene.data = foldchanges,
         pathway.id = "hsa04514",
         species = "hsa",
         kegg.native = FALSE,  # Use graphical output (non-native)
         out.suffix = "hsa04514_pdf")